In [10]:
import csv
import psycopg2
from psycopg2.extras import execute_batch
import uuid
from datetime import datetime

# 数据库连接参数
conn_params = {
    "dbname": "sora",
    "user": "sora",
    "password": "2ye2ei7XNeHA5nwK",
    "host": "47.117.170.179",  # 确保这里是正确的IP地址
    "port": "5432"  # 确保这里是正确的端口号
}


# 连接数据库
conn = psycopg2.connect(**conn_params)
cursor = conn.cursor()
# 打印数据库连接信息
print(f"Connected to {conn_params['dbname']} at {conn_params['host']}")

Connected to sora at 47.117.170.179


In [11]:
# 读取CSV文件
with open('videos_info.csv', 'r', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    videos_data = []
    for row in reader:
        # 为每个视频条目自动生成 UUID
        video_uuid = str(uuid.uuid4())
        # 使用当前时间作为 created_at 的值
        created_at = datetime.now()
        # 假设视频状态为 1（您可以根据实际情况调整）
        status = 1

        # 将从 CSV 读取的数据和自动生成的数据添加到列表中
        videos_data.append((
            video_uuid,  # user_uuid 字段，假设每个视频都有一个唯一的用户UUID
            row['video_description'],
            row['video_url'],
            row['poster_url'],
            row['poster_url'],  # 假设 CSV 文件中包含 post_url 字段
            row['user_nickname'],
            row['user_avatar_url'],
            created_at,
            video_uuid,  # 视频的 UUID
            status
        ))

# 执行批量插入到 videos 表
insert_query = """
INSERT INTO videos (
    user_uuid, video_description, video_url, cover_url, post_url, user_nickname, user_avatar_url, created_at, uuid, status
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
execute_batch(cursor, insert_query, videos_data)

# 提交事务并关闭连接
conn.commit()
cursor.close()
conn.close()

print("数据导入完成。")


数据导入完成。


In [ ]:
# 备选 优化
# 逐条执行插入到 videos 表
for video_data in videos_data:
    cursor.execute(insert_query, video_data)
    conn.commit()  # 提交每一次插入操作